In [1]:
from google.colab import drive
import json
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
reviews_file = open('/gdrive/My Drive/reviews3_large.txt', 'r')
reviews = json.loads(reviews_file.read())

In [0]:
DESC = 'description'
RATING = 'rating'

review_descriptions = []
review_ratings = []
for review in reviews:
  review_descriptions.append(review[DESC])
  review_ratings.append(review[RATING])

In [0]:
%tensorflow_version 2.x

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM,Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokens = []
for description in review_descriptions:
  tokens.extend(description.split(" "))

tokenizer = Tokenizer()

tokenizer.fit_on_texts(tokens)
review_descriptions_tokenized = tokenizer.texts_to_sequences(review_descriptions)

In [0]:
test_train_cutoff = len(review_descriptions_tokenized) - 500

X_train = review_descriptions_tokenized[0:test_train_cutoff]
y_train = review_ratings[0:test_train_cutoff]

X_test = review_descriptions_tokenized[test_train_cutoff:]
y_test = review_ratings[test_train_cutoff:]

In [0]:
# truncate and pad input sequences
max_review_length = 50
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [0]:
import numpy

def create_arr(index):
  arr = [0, 0, 0, 0, 0]
  arr[index - 1] = 1
  return arr


y_train = numpy.array([create_arr(int(x)) for x in y_train])
y_test = numpy.array([create_arr(int(x)) for x in y_test])

In [12]:
y_train

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [20]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(len(tokens), embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(300, input_shape=(len(X_train), embedding_vecor_length, max_review_length), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(300, input_shape=(400, 1, max_review_length)))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 32)            933664    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 300)           399600    
_________________________________________________________________
dropout (Dropout)            (None, 50, 300)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 1505      
Total params: 2,055,969
Trainable params: 2,055,969
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
56/56 [==============================] - 37s 664ms/step - loss: 0.2654 - accuracy: 0.8148
Epoch

In [18]:
print(len(X_test))
print(len(y_test))

500
500


In [21]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.40%
